Using https://www.pyimagesearch.com/2016/02/15/determining-object-color-with-opencv/ to work coordinates for tokens based on colour.

https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/ for downloading packages in Notebook.

In [3]:
# Install a pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install --upgrade imutils
# !{sys.executable} -m pip install --upgrade opencv-python

In [4]:
# import the necessary packages
import argparse
import imutils
import cv2
import numpy as np

In [5]:
# load the image, convert it to grayscale, blur it slightly,
# and threshold it
image = cv2.imread('board3.jpg')

https://www.pyimagesearch.com/2014/08/04/opencv-python-color-detection/ using to mask the image to only deal with parts of a certain colour.

In [6]:
lower = np.array([100, 0, 0], dtype = "uint8") # BGR
upper = np.array([255, 100, 100], dtype = "uint8")

# find the colors within the specified boundaries and apply
# the mask
mask = cv2.inRange(image, lower, upper)
image = cv2.bitwise_and(image, image, mask = mask)

In [7]:
# Each element has the form (x,y,area)
blobs = [];

In [8]:
# Our image is already white on black, but we apply grayscale
# again, blur the image for better detection and then apply
# a threshold.
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY)[1]

# Contours are a curve that joins all the continous points
# around an object with a specific colour intensity.
# in openCV this is finding white objects on a black background.
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)

# loop over the contours
for c in cnts:
    # compute the center of the contour
    M = cv2.moments(c) # Weighted average of pixel intensities
    
    # Computing the coordinates of the centre of each token.
    # Coordinates are relative to the image.
    # So they are on a 1200 by 720 grid.
    
    # Small amount of the colour picked up, ignore as no area so
    # will not be a token.
    if M["m00"] == 0: continue
    
    x = int(M["m10"] / M["m00"])
    y = int(M["m01"] / M["m00"])
    
    blobs = blobs + [(x,y,M["m00"])]
    print(str(M["m00"]))
 
    # Code off of the internet that can plot the contours.
    cv2.drawContours(image, [c], -1, (0, 255, 0), 2)
    cv2.circle(image, (x, y), 7, (255, 255, 255), -1)
    cv2.putText(image, "center", (x - 20, y - 20),
    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

28.0
192.5
2261.0
2100.5
7.0
6.5
1.0
17.5
4.5
3.0
116.0
42.0
2.5
86.5
332.0
114.0
0.5
1.5
0.5
1.0
47.5
10.0
3.0
512.5
1.5
722.0
768.0
26.0
92.0


In [9]:
def get_shapes(image,lower, upper):
    # From RGB to BGR
    lower.reverse()
    upper.reverse()
    
    lower = np.array(lower, dtype = "uint8")
    upper = np.array(upper, dtype = "uint8")
    
    # find the colors within the specified boundaries and apply the mask
    mask = cv2.inRange(image, lower, upper)
    image = cv2.bitwise_and(image, image, mask = mask)
    
    # Each element has the form (x,y,area)
    xs = [];
    ys = [];
    areas = []; 
    
    # Our image is already white on black, but we apply grayscale
    # again, blur the image for better detection and then apply
    # a threshold.
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY)[1]

    # Contours are a curve that joins all the continous points
    # around an object with a specific colour intensity.
    # in openCV this is finding white objects on a black background.
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    
    # loop over the contours
    for c in cnts:
        # compute the center of the contour
        M = cv2.moments(c) # Weighted average of pixel intensities
    
        # Computing the coordinates of the centre of each token.
        # Coordinates are relative to the image.
        # So they are on a 1200 by 720 grid.
    
        # Small amount of the colour picked up, ignore as no area so
        # will not be a token.
        if M["m00"] == 0: continue
        
        xs += [M["m10"] / M["m00"]]
        ys += [M["m01"] / M["m00"]]
        areas += [M["m00"]]
        
        
        
        x = int(M["m10"] / M["m00"])
        y = int(M["m01"] / M["m00"])
        
        # Code off of the internet that can plot the contours.
        cv2.drawContours(image, [c], -1, (0, 255, 0), 2)
        cv2.circle(image, (x, y), 7, (255, 255, 255), -1)
        cv2.putText(image, "center", (x - 20, y - 20),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    
    cv2.imshow("cam-test",image)
    cv2.waitKey(10)

    return np.row_stack((xs,ys,areas))

In [10]:
# Getting all of the black tokens
black = get_shapes(image,[0,0,140],[100,100,255])
# Getting all of the white tokens
white = get_shapes(image,[140,0,0],[255,100,100])

# Creating a matrix each column contains details of a token (x,y,area,colour)
black = np.row_stack(black,np.array(["B"] * np.shape[0]))
white = np.row_stack(white,np.array(["W"] * np.shape[0]))
tokens = np.hstack((black,white))

x_inds = np.digitize(tokens[0], 13 / 22 * np.arange(1,12))
y_inds = np.digitize(tokens[1], [8/22, 14/22])

knocked_out = []
board = [[]] * 24

for (s,x,y) in zip(tokens,x_inds,y_inds):
    if y == 1 or x == 6: # Middle section of the board
        knocked_out += [s]
    elif y == 0:
        board[x] += s
    else:
        board[23-x] += s

board

TypeError: 'function' object is not subscriptable

In [ ]:
while True:
    cam = cv2.VideoCapture(1)   # 0 -> index of camera
    retrieved, image = cam.read()
    #image = cv2.imread('board5.jpg')
    if retrieved:
        #shapes = get_shapes(image,[0,0,100],[100,100,255])
        shapes = get_shapes(image,[95,220,95],[180,255,180])
        #v2.imshow("cam-test",image)
        #cv2.waitKey(10)
    else:
        print("Error reading from webcam")
        cv2.destroyAllWindows()
        break

In [ ]:
x = np.array([-1, 0.5, 1.25, 2.75, 4.24, 19.25])
bins = np.array([0.0, 1.0, 2.5, 4.0, 10.0])
inds = np.digitize(x, bins)
inds

In [ ]:
cv2.imshow('board.jpg',image)
cv2.waitKey(0)
cv2.destroyAllWindows()